In [1]:
import speakeasy
import os
import time
import pandas as pd
import numpy as np
from pefile import PEFormatError
from unicorn import UcError

In [3]:
MALWARE_PATH = "../../../data/pe.dataset/PeX86Exe/ransomware/"
files_200 = [x for x in os.listdir(MALWARE_PATH)[200:400]]

In [3]:
reports = {}
timedeltas = []

now = time.time()
for i, file in enumerate(files_200):
    # calculations
    took = time.time()-now
    timedeltas.append(took)
    now = time.time()

    print(f"{i}/{len(files_200)}, took: {took:.2f}s", end=" | ")
    se = speakeasy.Speakeasy()
    try:
        module = se.load_module(MALWARE_PATH+file)
        se.run_module(module)
        reports[file] = se.get_report()
    except PEFormatError as ex:
        print(f"\nfailed {file}", ex)
    except UcError as ex:
        print(f"\nfailed {file}", ex)

print(f"\naverage analysis time per sample: {np.mean(timedeltas)}")

0/200, took: 0.00s | 1/200, took: 0.94s | 2/200, took: 0.38s | 3/200, took: 27.99s | 4/200, took: 0.32s | 5/200, took: 0.40s | 6/200, took: 1.04s | 7/200, took: 1.04s | 8/200, took: 0.45s | 9/200, took: 12.72s | 10/200, took: 0.91s | 11/200, took: 2.28s | 12/200, took: 0.53s | 13/200, took: 0.47s | 14/200, took: 0.86s | 15/200, took: 1.41s | 16/200, took: 0.52s | 17/200, took: 1.01s | 18/200, took: 5.48s | 19/200, took: 0.61s | 20/200, took: 1.08s | 21/200, took: 1.09s | 22/200, took: 0.50s | 23/200, took: 2.10s | 24/200, took: 6.16s | 25/200, took: 0.39s | 26/200, took: 60.36s | 27/200, took: 60.38s | 28/200, took: 1.64s | 29/200, took: 0.40s | 30/200, took: 2.04s | 31/200, took: 1.88s | 32/200, took: 1.10s | 33/200, took: 0.61s | 34/200, took: 1.19s | 35/200, took: 60.59s | 36/200, took: 1.05s | 37/200, took: 0.36s | 38/200, took: 0.33s | 39/200, took: 1.19s | 40/200, took: 1.47s | 41/200, took: 1.49s | 42/200, took: 1.00s | 43/200, took: 0.44s | 44/200, took: 0.90s | 45/200, took: 1

In [5]:
import pickle
# with open("reports_ransomware_200_400_nomods.pickle", "wb") as fhandle:
#     pickle.dump(reports, fhandle, protocol=pickle.HIGHEST_PROTOCOL)

with open("reports_ransomware_200_400_nomods.pickle", "rb") as fhandle:
    reports = pickle.load(fhandle)

In [6]:
def parse_report(report):
    aa = pd.json_normalize(report)
    ab = pd.json_normalize(aa["entry_points"].iloc[0])
    try:
        if ab["apis"].any():
            #print(ab["apis"])
            ac = pd.json_normalize(ab["apis"].iloc[0])
        else:
            ac = []
    except AttributeError:
        ac = []
    return aa, ab, ac

In [7]:
bs = []
for i,f in enumerate(files_200[0:10]):
    a, b, c = parse_report(reports[f])
    if "error.type" in b.keys():
        bs.append(b)
        print(f," : " ,b["error.type"].iloc[0])
    else:
        print(f, " : success")

916bc6fd4d12c7f53f81d4e3bdf21271ac70a928fab0c9622b61e0a4ac504c9d  :  Failed to get character width from function: kernel32.GetVolumeInformation
16815410c21e49d86db3ec362142116367a373b5e0e7d0ac5c6c568edb5b0567  :  invalid_fetch
7b500f44908f6a9abef119e71e5774a1769717fb26b934e30511fa9827cb9466  :  nan
d473941e52b9c094b93045309228f6d4d3749c16ccbb88bcca9c7123b1a4de27  :  unsupported_api
bc61e33eedd1a5f70730fd7d7d54842b694522912bf5a0fa5dba900586d3816e  :  unsupported_api
eee2e1596b91c478c34ce46819a810033c517861348b99806def31ac81811f0d  :  Failed to get character width from function: kernel32.GetVolumeInformation
ed55da050e6765f5aa3026e1549706e2541e5744d4f8fd7fa073c9493603772d  :  Failed to get character width from function: kernel32.GetVolumeInformation
37f5efd1aed9594306f7d26841829d6f11d0e73efc68eeda0ebde9ee1f1cfe75  :  unsupported_api
8a9afbb3536092092708cbe02e86cb40d5ce8952d4778a79e0fa5260f542712b  :  'Win32Emulator' object has no attribute 'dev_ioctl'
f562018b8ee371f1c2fe7b38263342e745d1

In [15]:
bs[0]["error.type"].iloc[0]

'Failed to get character width from function: kernel32.GetVolumeInformation'

## examine succeeded reports

In [91]:
suceeded_report = reports["f118b1f459eff5bfbf58eb2e6677c593ea47974c6f738e9c4b0fd4287940df40"]
suceeded_aa = pd.json_normalize(suceeded_report["entry_points"])
print(suceeded_aa.columns)
suceeded_aa

Index(['ep_type', 'start_addr', 'ep_args', 'apihash', 'apis', 'ret_val',
       'dynamic_code_segments', 'error.type', 'error.count', 'error.pc',
       'error.last_api'],
      dtype='object')


,ep_type,start_addr,ep_args,apihash,apis,ret_val,dynamic_code_segments,error.type,error.count,error.pc,error.last_api
0,module_entry,0x404663,"[0x4000, 0x4010, 0x4020, 0x4030]",c16d89e1464b49e5533e624f6b5137c2154a9c5dc95333...,"[{'pc': '0x409d4b', 'api_name': 'KERNEL32.GetS...",0x0,[],max_api_count,10000,0x401875,KERNEL32.GetThreadPriority


## examine failed reports

In [92]:
failed_report = reports[list(reports.keys())[0]]
failed_aa = pd.json_normalize(failed_report["entry_points"])
print(failed_aa.columns)
failed_aa

Index(['ep_type', 'start_addr', 'ep_args', 'apihash', 'apis', 'ret_val',
       'dynamic_code_segments', 'error.type', 'error.pc', 'error.address',
       'error.instr', 'error.regs.esp', 'error.regs.ebp', 'error.regs.eip',
       'error.regs.esi', 'error.regs.edi', 'error.regs.eax', 'error.regs.ebx',
       'error.regs.ecx', 'error.regs.edx', 'error.stack', 'error.traceback'],
      dtype='object')


,ep_type,start_addr,ep_args,apihash,apis,ret_val,dynamic_code_segments,error.type,error.pc,error.address,...,error.regs.ebp,error.regs.eip,error.regs.esi,error.regs.edi,error.regs.eax,error.regs.ebx,error.regs.ecx,error.regs.edx,error.stack,error.traceback
0,module_entry,0x650a99,"[0x4000, 0x4010, 0x4020, 0x4030]",e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,[],0x77000f93,[],Failed to get character width from function: k...,0x77000f93,0x77000f93,...,0x01212000,0x77000f93,0x00401d1f,0x00024c15,0x77000f93,0x01211fd8,0x00000003,0x01211fdc,[sp+0x00: 0x00401c77 -> emu.module.24c3804ea87...,"Traceback (most recent call last):\n File ""/u..."


In [95]:
error_cols = set(failed_aa) - set(suceeded_aa)   
error_cols

{'error.address',
 'error.instr',
 'error.regs.eax',
 'error.regs.ebp',
 'error.regs.ebx',
 'error.regs.ecx',
 'error.regs.edi',
 'error.regs.edx',
 'error.regs.eip',
 'error.regs.esi',
 'error.regs.esp',
 'error.stack',
 'error.traceback'}

In [99]:
failed_aa[error_cols]

,error.regs.esp,error.regs.edx,error.address,error.regs.edi,error.regs.ebx,error.instr,error.regs.eax,error.traceback,error.regs.ebp,error.stack,error.regs.ecx,error.regs.esi,error.regs.eip
0,0x01211fb4,0x01211fdc,0x77000f93,0x00024c15,0x01211fd8,"mov edi, edi",0x77000f93,"Traceback (most recent call last):\n File ""/u...",0x01212000,[sp+0x00: 0x00401c77 -> emu.module.24c3804ea87...,0x00000003,0x00401d1f,0x77000f93


In [103]:
print(failed_aa["error.traceback"].iloc[0])

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/windows/winemu.py", line 1168, in handle_import_func
    rv = self.api.call_api_func(mod, func, argv, ctx=default_ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/winapi.py", line 77, in call_api_func
    return func(mod, self.emu, argv, ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/usermode/kernel32.py", line 3587, in GetVolumeInformation
    cw = self.get_char_width(ctx)
  File "/usr/local/lib/python3.6/site-packages/speakeasy_emulator-1.5.9-py3.6.egg/speakeasy/winenv/api/api.py", line 343, in get_char_width
    raise ApiEmuError('Failed to get character width from function: %s' % (name))
speakeasy.errors.ApiEmuError: Failed to get character width from function: kernel32.GetVolumeInformation



In [96]:
success_cols =  set(suceeded_aa) - set(failed_aa)
success_cols

{'error.count', 'error.last_api'}

In [104]:
suceeded_aa[success_cols]

,error.last_api,error.count
0,KERNEL32.GetThreadPriority,10000
